In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from dotenv import load_dotenv
import pathlib
# from tqdm import tqdm
from tqdm.autonotebook import tqdm
from pathlib import Path
from pprint import pprint
import numpy as np
import hashlib

import polars as pl
from glob import glob

from openai import OpenAI
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

from pinecone_text.sparse import BM25Encoder


root_dir = Path(os.getcwd()).parent.parent
sys.path.insert(0, str(root_dir))

pl.Config.set_fmt_str_lengths(300)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(20);

C:\Users\edgarmp\AppData\Local\Temp\ipykernel_17884\3010044380.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from src.d01_data.data import (read_json, create_index_if_not_exists, upsert_vectors_in_batches,
                                delete_docs_by_id, get_all_id_prefixes, match_prefix, json_dump)

from src.d00_utils.utils import (get_tokens_len, generate_sparse_vector_in_batches,
                                 dict_to_document_boe, metadata_to_uuid)

load_dotenv('../../.env')
 
intermediate_path = root_dir / 'data' / '02_intermediate'
output_path = root_dir / 'data' / '04_model_output'

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
EMBEDDING_MODEL = os.getenv('EMBEDDING_MODEL')

In [13]:
pc = Pinecone(api_key=PINECONE_API_KEY)
pc_index = create_index_if_not_exists(client=pc, index_name='hackaton')
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL, dimensions=3072)

Index hackaton created


In [5]:
boe_files = glob(str(intermediate_path / '*/*.json'))
boe_files = [Path(f) for f in boe_files]

print(f'Número de ficheros boe: {len(boe_files)}')

Número de ficheros boe: 24


In [6]:
all_syllabus_docs = []
debug = True
for file in boe_files:
    tema = file.parent.name
    filename = file.name
    
    syllabus_data = read_json(file)
    chunks = list(syllabus_data.values())
    
    tokens = [get_tokens_len(art if tema != 'orientacion' else art['chunk_content']) for art in chunks]
    
    if debug:
        print(f'Procesando el fichero {str(Path(tema) / filename)}')
        print(f'Chunk más grande: {max(tokens)} tokens en índice {np.argmax(tokens)}')
        print(f'Chunk más pequeño: {min(tokens)} tokens en índice {np.argmin(tokens)}\n{chunks[np.argmin(tokens)]}\n')
    syllabus_docs = dict_to_document_boe(estructured_dict=syllabus_data,
                                        tema=tema, origen=filename)
    all_syllabus_docs.extend(syllabus_docs)
    
print(f'Número total de documentos {len(all_syllabus_docs)}')
    

Procesando el fichero boe\BOE-A-2000-544-consolidado_boe.json
Chunk más grande: 967 tokens en índice 38
Chunk más pequeño: 47 tokens en índice 51
BOE-A-2000-544-consolidado.pdf 
 TÍTULO II 
 CAPÍTULO III 
 Sección SIN ASIGNAR 
 Artículo 38 bis  Régimen especial de los investigadores: (Derogado).

Procesando el fichero boe\BOE-A-2011-7703-consolidado_boe.json
Chunk más grande: 1008 tokens en índice 264
Chunk más pequeño: 43 tokens en índice 118
BOE-A-2011-7703-consolidado.pdf 
 TÍTULO IV 
 CAPÍTULO IV 
 Sección SIN ASIGNAR 
 Artículo 77  Procedimiento: (Derogado)

Procesando el fichero extra\Convenio contra la tortura.json
Chunk más grande: 437 tokens en índice 11
Chunk más pequeño: 56 tokens en índice 20
Convenio contra la tortura.pdf 
 son igualmente auténticos, se depositará en poder del Secretario General de las 
Naciones Unidas.  
2. El Secretario General de las Naciones Unidas remitirá copias certificadas de la 
presente Convención a todos los Estados.

Procesando el fichero extra

In [7]:
# prefixes = get_all_id_prefixes(index=pc_index, level=1)
# prefixes

In [8]:
# matches_prefixes = match_prefix(index=pc_index, prefixes={'Código Penal.pdf', 'Codigo_Civil.pdf'})
# matches_prefixes

In [9]:
# removed_docs = delete_docs_by_id(index=pc_index, prefix='CE.pdf')

In [10]:
final_uuids = metadata_to_uuid(all_syllabus_docs)

assert len(final_uuids) == len(set(final_uuids)), 'Hay ids duplicados'
del final_uuids

In [11]:
bm25 = BM25Encoder().default()
bm25.fit([doc.page_content for doc in all_syllabus_docs])
bm25.dump(output_path / 'bm25_values.json')

100%|██████████| 1281/1281 [00:12<00:00, 105.70it/s]


In [14]:
vectors = generate_sparse_vector_in_batches(documents=all_syllabus_docs, embedding_model=embeddings,
                                            fitted_bm25=bm25, batch_size=64)
upsert_vectors_in_batches(vectors=vectors, index=pc_index, batch_size=100)